## Fossil Classification with Deep Learning (AA-AI Team Staff Conference)
The micro-fossil data for this ML model trianing is provided by the Paleontology team, GNS Science.

We will see in this notebook how to train and evaluate a deep convolutional neural network (CNN) to classify images of fossils sharing sometimes the same species but different genuses. We have selected a set of images for 14 different classes and made sure we have a sufficient number of images per class to start. 

### 1. Simple Convolutional Neural Network

Here, we will start by creating a simple CNN, train it on a subset of the data (the training dataset) and evaluate its performance on the rest of the dataset (training dataset). We will look at how it performs on some randomly picked images.

`nn_utils.py` is a Python script, where we coded all the functions necessary to this hands-on. **Let start!**


Loading the training data: upload the ImageClassesM.zip file

In [ ]:
from google.colab import files
print("Please upload the ImageClassesM.zip file")
_ =  files.upload()


Unzip the data

In [ ]:
!unzip ImageClassesM.zip > /dev/null

Install the machine learning package we ill be using

In [ ]:
# #specify the tensorflow version
%tensorflow_version 2.x
import tensorflow as tf
print(tf.__version__)
from tensorflow import keras
!pip install -q -U keras-tuner

In [ ]:
print("Please upload the nn_utils.py file.")
_ =  files.upload()

In [ ]:
import nn_utils 
# -- function to split a dataset into a training and a testing dataset
from sklearn.model_selection import train_test_split


######Now that the modules are loaded, we are going to import the images and store them into a `Numpy` array along a class array which will be the ground truth. 

In [ ]:
# -- read data, format images into an Array and encode the classes
input_dir = "/content/ImageClassesM"
ImageArray, Labels = nn_utils.read_images(images_dir=input_dir)

Now it is time to split the dataset into *training* and *testing*. Something to keep in mind is that this dataset like many other is not balanced, there are more images of certain classes than others. If not taken into account this could lead to a biased model. When spliting the dataset, we will use *Stratified Sampling* to ensure that each class within the dataset receives a statistically accurate representation within both training and testing subsets.

In [ ]:
X_train, X_test, y_train, y_test = \
    train_test_split(ImageArray, Labels, test_size=0.2, random_state=42,
        stratify=Labels)

#####We can now create our simple CNN model and use the training dataset to teach the model to recognize fossils. During training, it is common to keep a small portion of the training dataset to *validate* or test the model as it is training. This always ensures that the model's accuracy is tested on never-seen-before data. The `batch_size` controls the number of images (in this case) fed to the model at a time and the number of `epochs` is the number of times it is trained on the training dataset. Let's also plot a summary of the training to see how it went.

In [ ]:
image_shape = ImageArray.shape[1:] # dimension of one image -> input
number_of_class = y_train.shape[1] # number of class, size of the prediction distribution -> output

model, summary = nn_utils.CNNbase_model(
        input_shape=image_shape, n_class=number_of_class)

# -- training and evaluating the model with a subset of unseen data
history = model.fit(
    x=X_train, y=y_train, batch_size=32, validation_split=0.2, epochs=25,
)

nn_utils.summarize_diagnostics(history=history, filename="/content/base_summary.jpg")

In [ ]:
eval_result = model.evaluate(x=X_test, y=y_test)
print(f"""
        Evaluation of test data [test loss, test accuracy] is {eval_result}.
       """)

# -- try on some examples
X_sample, y_sample = nn_utils.sample_dataset(X=X_test, y=y_test, size=9, random_state=42)
y_pred = model.predict(x=X_sample)
nn_utils.plot_prediction(x=X_sample, y_true=y_sample, y_pred=y_pred, filename="/content/base_prediction.jpg")


Often the first model (CNN) is not performing well. This could be due to multiple factors, e.g. the training dataset is too small, the architecture of the model is too simple, etc..

### 2. Model optimization and hyperparameters tunning

We know that one of the limitation of the previous model is its architecture. Optimizing the architecture and tunning the hyperparameters (*hypertuning*) of a Machine Learning model is almost an art by itself. You can find more details on `Tensorflow`'s tuner [here!](https://www.tensorflow.org/tutorials/keras/keras_tuner).

Here, the parameters that will be optimized are for example the size of the window to perform the 2D convolution in the first half of the model or the number of hidden layer in the second half of the model. 

In [ ]:
image_shape = ImageArray.shape[1:] # dimension of one image -> input
number_of_class = y_train.shape[1] # number of class, size of the prediction distribution -> output

tuner = nn_utils.hyper_tuning(input_shape=image_shape, n_class=number_of_class)
tuner.search(X_train, y_train, validation_split=0.2)
tuner.results_summary()

best_hps = tuner.get_best_hyperparameters(1)[0]
print(f"""
        The hyperparameter search is complete. The optimal number of
        layers {best_hps.get('num_layers')} and the optimal learning rate for the optimizer
        is {best_hps.get('learning_rate')}. Best Conv. Filter: {best_hps.get('conv_filter')}. 
        Best Conv. Kernel: {best_hps.get('conv_kernel')}. 
    """)

Once the tuner is done, we get a set of hyperparameters that correspond to the best results on the validation dataset, which is here 30% of the training dataset. We can then train a model with these hyperparameters and evaluate our new model! 

In [ ]:
best_model = tuner.hypermodel.build(best_hps)
history = best_model.fit(
    x=X_train, y=y_train, batch_size=32, validation_split=0.3, epochs=50)

# -- 3.3 testing the model
eval_result = best_model.evaluate(x=X_test, y=y_test)
print(f"""
            Evaluation of test data [test loss, test accuracy] is {eval_result}.
        """)

# -- plot summary 
nn_utils.summarize_diagnostics(history=history, filename="/content/tuned_summary.jpg")

Let's look at some examples..

In [ ]:
X_sample, y_sample = nn_utils.sample_dataset(X=X_test, y=y_test, size=9, random_state=42)
y_pred = best_model.predict(x=X_sample)
nn_utils.plot_prediction(x=X_sample, y_true=y_sample, y_pred=y_pred, filename="/content/tuned_prediction.jpg")


Still not very convincing, right? Remember it might not be **just** the architecture, most likely our training dataset is too limited. We will see in the next exercise how to use a pre-trained model and transfer learning to have a highly accurate model to classify our images.

### 3. Transfer learning and how to use a pre-trained model

One other limitation of today's exercises are that we are limited in the number of images that we have. Fortunately for us, a lot of people have trained many other models for image classification. While there might not be a specific model trained to classify images of fossils we can still make use of a model trained on all sort of images. For example, we are going to use the **ResNet50** model a CNN 50-layers deep model trained on more than a million of labelled images such as ballons or strawberries from https://www.image-net.org/. To be able to use it for classifying fossils, we are going to use a principle called *Transfer Learning*, we are going to use a model trained on some data and apply it on some other data. The only work we have to do is to make sure that it is able to do what we want so we simply need to training a little bit (for example 5 `epochs`) on our training dataset. 

In practice, it consits in *replacing* the last part of the CNN, the fully connected layers which does the classification based on the features extracted by the convolutional layers by new fully connected layers that need to be trained.

In [ ]:
# -- 3.1 init of the Model with new top layer
image_shape = ImageArray.shape[1:] # dimension of one image -> input
number_of_class = y_train.shape[1] # number of class, size of the prediction distribution -> output
model = nn_utils.CNNtrained_model(input_shape=image_shape, n_class=number_of_class)

# -- 3.2 training and evaluating the model with a subset of unseen data (validation)
history = model.fit(x=X_train, y=y_train, batch_size=32, validation_split=0.2, epochs=25)

# -- 3.3 testing the model
eval_result = model.evaluate(x=X_test, y=y_test)
print(f"""
            Evaluation of test data [test loss, test accuracy] is {eval_result}.
        """)

# -- 3.4 plot summary 
nn_utils.summarize_diagnostics(history=history, filename="/content/pretrained_summary.jpg")

It's a win! By using Transfer Learning, we clearly see an improvement. Let's have a look at some examples.

In [ ]:
# -- 3.5 Show some examples
X_sample, y_sample = nn_utils.sample_dataset(X=X_test, y=y_test, size=9, random_state=42)
y_pred = model.predict(x=X_sample)
nn_utils.plot_prediction(x=X_sample, y_true=y_sample, y_pred=y_pred, filename="/content/pretrained_prediction.jpg")

Not a single mistake! 
This new model to classify images of fossils is much more accurate than our first models thanks to the **ResNet50** pre-trained model. In practice, we benefit from the already very well trained filters that are in the convolutional layers that extract meaningful and accurate features which ease the classification for the fully connected layers. In fact, Transfer Learning is becoming common: [a good example](https://agupubs.onlinelibrary.wiley.com/doi/full/10.1029/2021JB021910?utm_sq=gqzx3u0l8q) is a model trained to pick seismic wave arrivals in Southern California which was use to create a model specifically trained for picking events at the Nabro Volcano, a Stratovolcano in Eritrea. 

### Conclusion 

We saw in this workshop how easy it is to create, train and evaluate a machine learning model. We also saw through the classification of fossils that it is not so easy to have an accurate model when our dataset is limited or when the architecture of our model is not optimized. 
